In [3]:
%%capture
%load_ext autoreload
%autoreload 2
%cd /home/anhvth5/hypersloth

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [2]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0' # Choose any GPU you want to use
# import torch
# from unsloth import FastModel


# fourbit_models = [
#     # 4bit dynamic quants for superior accuracy and low memory use
#     "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
#     "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
#     "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
#     "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

#     # Other popular models!
#     "unsloth/Llama-3.1-8B",
#     "unsloth/Llama-3.2-3B",
#     "unsloth/Llama-3.3-70B",
#     "unsloth/mistral-7b-instruct-v0.3",
#     "unsloth/Phi-4",
# ] # More models at https://huggingface.co/unsloth

# model, tokenizer = FastModel.from_pretrained(
#     model_name = "unsloth/Qwen2.5-1.5B-Instruct",
#     max_seq_length = 16_000, # Choose any for long context!
#     load_in_4bit = True,  # 4 bit quantization to reduce memory
#     load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
#     full_finetuning = False, # [NEW!] We have full finetuning now!
# )

In [4]:
from HyperSloth.transformer_trainer_setup import setup_model_and_training
from HyperSloth.hypersloth_config import *

# Main configuration using Pydantic models
hyper_config_model = HyperConfig(
    grad_dir="/dev/shm/hypersloth",
    data=DataConfig(
        dataset_name_or_path="data/sharegpt/train_233k.json",
        num_samples=1000,
        group_by_length=True
    ),
    training=TrainingConfig(
        gpus=range(2),  # Change this to the number of GPUs you have
        loss_type="all",  # all or response_only, the loss will only be calculated on the response part of the input
    ),
    fast_model_args=FastModelArgs(
        model_name="unsloth/qwen2.5-0.5b-instruct",
        max_seq_length=16_000,
    ),
    lora_args=LoraArgs(
        r=16,
        lora_alpha=16,
    ),
)

# Training arguments using Pydantic model
training_config_model = TrainingArgsConfig(
    output_dir="saves/loras/250312/LC_EN_VI_TH_27B_233k/",
    per_device_train_batch_size=1,  #
    gradient_accumulation_steps=16,  # More GA help to reduce total communication time
    learning_rate=0.0002,
    per_device_eval_batch_size=2,
    eval_steps=100000,
    logging_steps=1,
    report_to="tensorboard",
    num_train_epochs=1,
    lr_scheduler_type="linear",
    warmup_steps=5,
    seed=42,
    save_total_limit=2,
    bf16=True,
    fp16=False,
    optim="adamw_8bit",
    weight_decay=0.01,
    packing=False,
)


output = setup_model_and_training(0, hyper_config_model, training_config_model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.16: Fast Qwen2 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA L40S. Num GPUs = 8. Max memory: 44.403 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/950 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/50 [00:00<?, ? examples/s]

2025-03-19 16:40:59.019 | INFO     | HyperSloth.transformer_trainer_setup:setup_model_and_training:90 - Model setup complete for GPU 0
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



=== EXAMPLE #1 ===
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
{
  "source_language": "Chinese",
  "target_language": "Vietnamese",
  "metadata": {
    "game_name": "LC_GS1_JX1M_VI"
  },
  "glossary": [
    {
      "source": "峨眉女",
      "target": "Nga Mi Nữ"
    },
    {
      "source": "10阶",
      "target": "Bậc 10"
    },
    {
      "source": "峨眉",
      "target": "Nga Mi"
    },
    {
      "source": "鞋子",
      "target": "Giày"
    },
    {
      "source": "普通",
      "target": "Thường"
    }
  ],
  "examples": [
    {
      "source": "10阶吃鸡鞋子",
      "target": "Giày Tử Chiến Bậc 10"
    }
  ],
  "text_to_translate": "[普通]10阶:峨眉女鞋子"
}<|im_end|>
<|im_start|>assistant
{
  "translation": "[Thường]Giày Nga Mi (Nữ) Bậc 10"
}<|im_end|>
<|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|><|vision_pad|

In [8]:
output1 = setup_model_and_training(1, hyper_config_model, training_config_model)

==((====))==  Unsloth 2025.3.16: Fast Qwen2 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA L40S. Num GPUs = 8. Max memory: 44.403 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
for i1, i2 in zip(output.train_dataset, output1.train_dataset):
    print(len(i1['input_ids']), len(i2['input_ids']))

273 274
245 246
127 128
136 141
404 408
440 440
259 259
186 186
143 144
313 315
183 184
166 166
516 516
235 235
93 93
184 184
265 266
261 261
374 375
374 374
310 310
545 546
434 436
223 223
323 324
927 938
356 356
805 809
263 264
184 185
218 218
312 313
249 249
168 169
607 607
497 499
402 402
180 180
665 666
514 514
244 245
508 508
278 279
288 288
489 490
567 569
190 190
121 123
686 688
87 88
413 415
428 429
193 194
88 88
89 89
333 333
153 153
292 293
132 133
1273 1283
481 482
225 225
240 240
243 243
395 395
675 676
286 286
401 402
409 410
256 256
89 90
88 89
1183 1214
297 297
195 196
343 343
208 208
353 354
97 97
169 170
677 679
120 120
342 343
381 381
179 180
343 344
285 285
226 227
287 287
315 316
335 336
1035 1040
1598 1611
522 526
216 216
505 506
372 373
284 285
289 292
162 162
331 331
276 276
521 521
227 228
101 117
317 318
1437 1453
526 527
215 216
1691 1732
280 281
161 161
318 319
237 237
163 165
761 766
256 257
420 420
361 361
982 994
187 187
652 652
235 236
321 321
123 123
36

: 

In [4]:
from pyexpat.errors import messages
from typing import Any
from speedy_utils.all import *
from datasets import load_dataset
from unsloth.chat_templates import standardize_data_formats
from datasets import Dataset


def get_chat_dataset(
    dataset_name: str, split: str = None, num_samples: int=None, tokenizer: Any = None
) -> Any:
    """
    Load and preprocess the dataset.

    Args:
        dataset_name (str): The name of the dataset to load.
        split (str): The dataset split to load.
        num_samples (int): The number of samples to select from the dataset.

    Returns:
        Any: The preprocessed dataset.
    """

    if os.path.exists(dataset_name):
        dataset = Dataset.from_json(dataset_name)
    else:
        dataset = load_dataset(dataset_name, split=split)
    dataset = standardize_data_formats(dataset)

    def apply_chat_template(examples):
        messages_key = "messages" if "messages" in examples else "conversations"
        texts = tokenizer.apply_chat_template(examples[messages_key], tokenize=False)
        return {"text": texts}
    if num_samples:
        num_samples = min(num_samples, len(dataset))
        dataset = dataset.shuffle(seed=42)
        dataset = dataset.select(range(num_samples))
    if tokenizer:
        dataset = dataset.map(apply_chat_template, batched=True)
    return dataset


# Example usage
dataset = get_chat_dataset("./data/cod_6k5.json", tokenizer=tokenizer)

In [5]:



from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_num_proc=4,
        dataset_text_field = "text",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 1000,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
print(tokenizer.decode(trainer.train_dataset[0]['input_ids']))

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user",
    response_part = "<|im_start|>assistant",
)

In [ ]:
trainer_stats = trainer.train()

In [11]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.643 GB.
2.645 GB of memory reserved.


In [ ]:
# model.save_pretrained('./outputs/lora/Qwen2.5-1.5B-Instruct-LORA-MATH')
# model.save_pretrained_merged('./outputs/lora/Qwen2.5-1.5B-Instruct-LORA-MATH-merged', tokenizer=tokenizer)

Unsloth: Merging weights into 16bit:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

In [ ]:
# from llm_utils import get_conversation_one_turn


# item = dataset[0]
# # messages = item["messages"][:-1]
# messages = get_conversation_one_turn(None, 'hoàng sa của nước nào, hãy trả lời bằng tiếng việt')

# text = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt = True, # Must add for generation
#     tokenize=False,nbs/test_unsloth.ipynbnbs/test_unsloth.ipynb˛˛
# )
# text += '<think>\n'


### VLLM INFerence